# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import yaml
import warnings
import numpy as np
import time
import torch
import pickle as pkl
from importlib import reload
import cv2

# plotting
import matplotlib.pyplot as plt
import utils

# dimensionality reduction
from sklearn.manifold import trustworthiness
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neighbors import LocalOutlierFactor
import umap

# Loading Image Paths and Feature Vectors

In [ ]:
# Load configuration
with open("config.yaml", 'r') as f:
        config = yaml.safe_load(f)

# Set base directory path
base_dir = config["project_paths"]["base_dir"]

# Images
image_dir = config["project_paths"]['images'] # path to the images
filenames = os.listdir(image_dir) # list of all images in the directory

# Sprite Images
sprite = os.path.join(base_dir, 'outputs/sprite_image.png')

# Feature Vector Dictionaries (img_filename:feature_vector)
feature_vector_path = os.path.join(base_dir, 'data/feature_vectors_dict_DM_full_dino_48_augmentations_05092024.pt')
dino_dict = torch.load(feature_vector_path)

# model name
model_name = 'dino'

# dictionary of feature_vectors dictionaries - {img_filename:feature_vector}
model_dict = {'dict': dino_dict,
                   'filenames': filenames,
                   'image_dir': image_dir,
                   'sprite': sprite
                  }

# getting dictionary of feature vectors in order of filenames
fv = {} # feature vectors

feat_dict = model_dict['dict']
vectors = [feat_dict[filename] for filename in model_dict['filenames'] if filename.endswith(('.jpg', '.jpeg'))]

fv[model_name] = vectors

# L2 normalization
fv["l2"] = preprocessing.normalize(fv["dino"], norm='l2')

# Make Sprite

In [ ]:
image_size = (50, 50)  # Size to which each image will be resized

#sprite = os.path.join(base_dir, 'outputs/sprite_image_150.png')
#utils.create_sprite(filenames, image_dir, sprite, image_size)

# Dimensionality Reduction

## PCA

In [ ]:
fv_reduced = {}
fv_whitened = {}
fv_explained_variance = {}

variance_threshold = [0.9]

# Print Individual Explained Variance Graphs
for name, vectors in fv.items():
        print(name)
        print('Number of feature vectors:', len(vectors))
        print(np.array(vectors).shape)

        for variance in variance_threshold:

                (vectors_reduced,
                vectors_whitened,
                explained_variance,
                num_components) = utils.print_pca_variations(vectors,
                                                        variance_threshold=variance,
                                                        show=True,
                                                        ratio=False)

                variance_name = '0_' + str(np.round(variance, 2)).split('.')[-1]

                fv_reduced[name] = vectors_reduced
                fv_whitened[name] = vectors_whitened
                fv_explained_variance[name] = explained_variance

print(num_components)

plt.figure(figsize=(8, 4))
for idx, (name, explained_variance) in enumerate(fv_explained_variance.items()):
        cumulative_variance = np.cumsum(explained_variance)  # Compute the cumulative sum of explained variance ratios

        plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance * 100, where='mid', label=f'DINOv2 Dreamachine Features')
        plt.axhline(y=variance_threshold[0]*100, color='green', label=f"Variance Threshold: {variance_threshold[0] * 100:.1f}%")
        plt.axvline(x=num_components, ymax=variance_threshold[0], color='purple', ls='--', label=f'Components Retained: {num_components}')
        plt.ylim(0, 100)
        plt.xlim(0, 1024)
        plt.grid(True)
        plt.legend(loc='lower right')
        plt.xlabel('principal components')
        plt.ylabel('percent variance explained')

print(fv_reduced.keys())

## UMAP

### Evaluating Trustworthiness looking at Components

In [ ]:
def evaluate_umap_embeddings(
        vectors,
        components,
        n_neighbors_range,
        trustworthiness_nn,
        metric="cosine",
        random_state=None):

    for n_neighbors in n_neighbors_range:

        print("NN:", n_neighbors)
        print(50*"-")

        plt.figure(figsize=(8, 5))
        plt.ylabel(f"Trustworthiness")
        plt.xlabel("UMAP Components")
        plt.title("UMAP embeddings: n_neighbors=" + str(n_neighbors))
        plt.grid(True)

        for trust_nn in trustworthiness_nn:

            print("trust n_neighbors:", trust_nn)

            embeddings, trust_scores = {}, {}

            for c in components:

                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")

                    umap_model = umap.UMAP(
                        n_components=c,
                        n_neighbors=n_neighbors,
                        min_dist=0.0,
                        metric=metric,
                        random_state=random_state,
                    ).fit_transform(vectors)

                embeddings[c] = umap_model
                trust_scores[c] = trustworthiness(np.asarray(vectors), umap_model, n_neighbors=trust_nn)

            plt.plot(list(trust_scores.keys()),
                    list(trust_scores.values()),
                    marker="o", label=trust_nn)

        plt.legend(title="Trustworthiness Parameter: NN")
        plt.show()


vectors = fv["l2"]

print(vectors.shape)

evaluate_umap_embeddings(vectors,
                        components=range(2, 10),
                        trustworthiness_nn=[25, 50, 75, 100, 125, 150],
                        n_neighbors_range=[25, 50, 75, 100, 125, 150])

### Evaluating Trustworthiness looking at NNeighbors (UMAP Parameter)

In [ ]:
def evaluate_umap_embeddings(
        vectors,
        components,
        n_neighbors_range,
        trustworthiness_nn,
        metric="euclidean",
        random_state=None):

    plt.figure(figsize=(8, 5))
    plt.ylabel(f"Trustworthiness")
    plt.xlabel("n_neighbors")
    plt.title("UMAP embeddings")
    plt.grid(True)

    for trust_nn in trustworthiness_nn:

        embeddings, trust_scores = {}, {}

        for n_neighbors in n_neighbors_range:

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")

                umap_model = umap.UMAP(
                    n_components=components,
                    n_neighbors=n_neighbors,
                    min_dist=0.0,
                    metric=metric,
                    random_state=random_state,
                ).fit_transform(vectors)

            embeddings[n_neighbors] = umap_model
            trust_scores[n_neighbors] = trustworthiness(np.asarray(vectors), umap_model, n_neighbors=trust_nn)

        plt.plot(list(trust_scores.keys()),
                list(trust_scores.values()),
                marker="o", label=trust_nn)

    plt.legend(title="Trustworthiness Parameter: NN")
    plt.show()

evaluate_umap_embeddings(fv["l2"],
                        components=6,
                        trustworthiness_nn=[25, 50, 75, 100, 125, 150],
                        n_neighbors_range=[25, 50, 75, 100, 125, 150])

### UMAP Projection

In [ ]:
vectors = fv["l2"]

"""# Save UMAP embeddings (6D, 100NN)
umap_model = umap.UMAP(
    n_components=6,
    n_neighbors=100,
    min_dist=0.0,
    random_state=42,
    metric="cosine",
    ).fit_transform(vectors)

umap_2d = umap.UMAP(
    n_components=2,
    n_neighbors=100,
    min_dist=0.0,
    metric="cosine",
    random_state=42,
    ).fit_transform(vectors)"""

umap_2d_150 = umap.UMAP(
    n_components=2,
    n_neighbors=150,
    min_dist=0.0,
    metric="cosine",
    random_state=42,
    ).fit_transform(vectors)

path_to_save = base_dir + "/outputs"
"""
filename = "umap_embeddings_6d_100nn.pkl"
with open(os.path.join(path_to_save, filename), "wb") as file:
    pkl.dump(umap_model, file)

filename = "umap_embeddings_2d_100nn.pkl"
with open(os.path.join(path_to_save, filename), "wb") as file:
    pkl.dump(umap_2d, file)"""

filename = "umap_embeddings_2d_150nn.pkl"
with open(os.path.join(path_to_save, filename), "wb") as file:
    pkl.dump(umap_2d_150, file)

## Visualizing Projections

### PCA Projection

In [ ]:
def print_pca_projection(vectors, model_dict, show_images):

        pca = PCA(n_components=2)
        vectors_pca = pca.fit(np.array(vectors)).transform(np.array(vectors))

        if show_images:
                utils.plot_with_images(vectors_pca,
                                       model_dict)
        else:
                plt.figure(figsize=(10, 10))
                plt.scatter(vectors_pca[:, 0], vectors_pca[:, 1])
                plt.xlabel('PCA Dimension 1')
                plt.ylabel('PCA Dimension 2')
                plt.title('PCA Visualization')
                plt.show()

        return vectors_pca

pca_projection = {}

for name, vectors in fv.items():
    print(name)

    pca_projection[name] = print_pca_projection(vectors, model_dict, show_images=True)

# Clustering

## HDBSCAN



In [ ]:
def plot_umap(vectors,
              model_dict,
              plot2d = True,
              show_images=False,
              zoom=0.3,
              n_neighbors=50,
              min_dist=0.0,
              metric="cosine",
              clusters=None,
              figsize=(30,30),
              subsample_size=5000
              ):

    umap_2d = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, metric=metric)
    X_umap_2d = umap_2d.fit_transform(np.array(vectors))

    if plot2d:
        print(f"Trustworthiness: {trustworthiness(vectors, X_umap_2d, n_neighbors=30):.2f}")

        if show_images:
            utils.plot_with_images(X_umap_2d, model_dict, figsize=figsize, zoom=zoom, cleaned=False, subsample_size=subsample_size)

        else:
            plt.figure(figsize=(20, 20))
            plt.scatter(X_umap_2d[:, 0], X_umap_2d[:, 1])
            plt.xlabel('UMAP Dimension 1')
            plt.ylabel('UMAP Dimension 2')
            plt.title('UMAP Visualization')
            plt.show()

    return X_umap_2d

### Grid Search

In [ ]:
import joblib
from sklearn.cluster import HDBSCAN
import hdbscan
from hdbscan.validity import validity_index
from tqdm import tqdm
from collections import defaultdict

vectors = fv_reduced["l2"]

clusters_dict = defaultdict(lambda: defaultdict(dict))  # {Cluster count: {n_neighbors: {min_samples: (cluster_labels)}}}

umap2d = plot_umap(vectors,
                model_dict,
                show_images=True,
                zoom=0.3,
                n_neighbors=100,
                min_dist=0,
                clusters=None,
                figsize=(30,30),
                subsample_size=5000
    )

for n_neighbours in [100]:

    print('n_neighbours: ', n_neighbours)

    umap_model = umap.UMAP(
    n_components=6,
    n_neighbors=n_neighbours,
    min_dist=0,
    random_state=42,
    metric="cosine"
    ).fit_transform(vectors)

    # umap_embedding is the 2‑D array returned by UMAP
    cache_dir = 'data/hdbscan_cache'
    mem = joblib.Memory(location=cache_dir, verbose=0)

    umap_model = np.asarray(umap_model, dtype=np.float64)

    outlier_size, num_clusters, dbcv = {}, {}, {}
    max_clusters = 0
    min_outliers = len(vectors)

    min_cluster_size = 99

    for min_samples in tqdm(range(1, 100, 1)):
        hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples,
                                        metric="cosine", algorithm="generic", memory=mem).fit(umap_model)
        outlier_size[min_samples+1] = np.sum(hdbscan_model.labels_ == -1)
        num_clusters[min_samples+1] = len(set(hdbscan_model.labels_)) - 1

        # Compute DBCV
        dbcv[min_samples+1] = validity_index(umap_model, hdbscan_model.labels_, metric="cosine")

        # Adding to parameter dictionary for stability comparision
        clusters_dict[num_clusters[min_samples+1]][n_neighbours][min_samples+1] = hdbscan_model.labels_

        # Finding Optimal Solution
        if num_clusters[min_samples+1] > max_clusters:
            max_clusters = num_clusters[min_samples+1]
            min_outliers = outlier_size[min_samples+1]
            best_min_samples = min_samples+1
        elif num_clusters[min_samples+1] == max_clusters:
            if outlier_size[min_samples+1] < min_outliers:
                min_outliers = outlier_size[min_samples+1]
                best_min_samples = min_samples+1

    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=best_min_samples, metric="cosine").fit(umap_model)

    print('Max clusters: ', max_clusters)
    print('Min outliers: ', min_outliers)

    print('Max/min # cluster: ', np.max(list(num_clusters.values())), np.min(list(num_clusters.values())))
    print('Max/min outlier size: ', np.max(list(outlier_size.values())), np.min(list(outlier_size.values())))

    figure = plt.figure(figsize=(12, 12))
    plt.subplot(3, 1, 1)
    plt.plot(outlier_size.keys(), outlier_size.values(), marker='o')
    plt.axvline(x=22, color='red', linestyle='--', label="Selected Solution: \n min_sample=22")
    plt.ylabel('Unclustered (Noise) Data Points')
    plt.legend()
    plt.grid()

    plt.subplot(3, 1, 2)
    plt.plot(num_clusters.keys(), num_clusters.values(), marker='o')
    plt.axvline(x=22, color='red', linestyle='--')
    plt.ylabel('Number of Clusters')
    plt.grid()

    plt.subplot(3, 1, 3)
    plt.plot(dbcv.keys(), dbcv.values(), marker='o')
    plt.axvline(x=22, color='red', linestyle='--')
    plt.xlabel('Min Samples')
    plt.ylabel('DBCV Score')
    plt.grid()

    plt.legend()
    plt.show()

    utils.print_projection(hdbscan_model.labels_, model_dict, umap2d, show_images=False, clusters_per_marker=None, zoom=0.4)
    utils.print_projection(hdbscan_model.labels_, model_dict, umap2d, show_images=True, clusters_per_marker=None, zoom=0.4)


### Clustering Comparisons

In [ ]:
from collections import defaultdict
from itertools import combinations
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

def _jaccard_coassignment(labels_a, labels_b, ignore_noise=True, noise_label=-1):
    """
    Jaccard index over *pairwise co-assignment*:
    J = |pairs clustered together in both| / |pairs clustered together in either|
    Optionally ignores any pair where at least one element is labeled as noise.
    """
    labels_a = np.asarray(labels_a)
    labels_b = np.asarray(labels_b)
    n = labels_a.shape[0]
    # mask for non-noise items if requested
    if ignore_noise:
        ok = (labels_a != noise_label) & (labels_b != noise_label)
    else:
        ok = np.ones(n, dtype=bool)
    idx = np.where(ok)[0]
    if len(idx) < 2:
        return np.nan  # not enough points to form pairs

    # For efficiency, compute co-assignment via label equality on an index subset
    la = labels_a[idx]
    lb = labels_b[idx]
    # co-assignment boolean matrices (upper triangle implied)
    same_a = la[:, None] == la[None, :]
    same_b = lb[:, None] == lb[None, :]
    # use upper triangle without diagonal
    iu = np.triu_indices(len(idx), k=1)
    A = same_a[iu]
    B = same_b[iu]
    inter = np.sum(A & B)
    union = np.sum(A | B)
    return np.nan if union == 0 else inter / union

def collect_solutions_for_K(clusters_dict, K):
    """
    clusters_dict structure:
      {cluster_count: {n_neighbors: {min_samples: labels_ndarray}}}
    Returns list of (n_neighbors, min_samples, labels) for the specified K.
    """
    out = []
    if K not in clusters_dict:
        return out
    for nn, inner in clusters_dict[K].items():
        for ms, labels in inner.items():
            out.append(((nn, ms), np.asarray(labels)))
    return out

def pairwise_metrics_for_K(clusters_dict, K, ignore_noise=True):
    sols = collect_solutions_for_K(clusters_dict, K)
    records = []
    for (key_i, lab_i), (key_j, lab_j) in combinations(sols, 2):
        ari = adjusted_rand_score(lab_i, lab_j)
        nmi = normalized_mutual_info_score(lab_i, lab_j)
        jac = _jaccard_coassignment(lab_i, lab_j, ignore_noise=ignore_noise)
        records.append({
            "K": K,
            "ref": key_i,
            "cmp": key_j,
            "ARI": ari,
            "NMI": nmi,
            "Jaccard_coassign": jac
        })
    df = pd.DataFrame.from_records(records)
    return df.sort_values(["ARI", "NMI"], ascending=False, ignore_index=True)

def reference_vs_others_for_K(clusters_dict, K, ref_key, ignore_noise=True):
    """
    ref_key = (n_neighbors, min_samples) of your chosen solution.
    Returns DataFrame comparing reference to all other solutions at K.
    """
    sols = dict(collect_solutions_for_K(clusters_dict, K))  # {(nn, ms): labels}
    if ref_key not in sols:
        raise ValueError(f"Reference key {ref_key} not found for K={K}.")
    lab_ref = sols[ref_key]
    rows = []
    for key, lab in sols.items():
        if key == ref_key:
            continue
        ari = adjusted_rand_score(lab_ref, lab)
        nmi = normalized_mutual_info_score(lab_ref, lab)
        jac = _jaccard_coassignment(lab_ref, lab, ignore_noise=ignore_noise)
        rows.append({"K": K, "ref": ref_key, "cmp": key, "ARI": ari, "NMI": nmi, "Jaccard_coassign": jac})
    return pd.DataFrame(rows).sort_values(["ARI", "NMI"], ascending=False, ignore_index=True)

def summarize_metrics(df):
    """
    Takes either the pairwise or reference-vs-others DataFrame and returns
    mean, median, and IQR for each metric.
    """
    def iqr(x):
        return np.nanpercentile(x, 75) - np.nanpercentile(x, 25)
    summary = {
        "mean_ARI": df["ARI"].mean(),
        "median_ARI": df["ARI"].median(),
        "IQR_ARI": iqr(df["ARI"]),
        "mean_NMI": df["NMI"].mean(),
        "median_NMI": df["NMI"].median(),
        "IQR_NMI": iqr(df["NMI"]),
        "mean_Jaccard": df["Jaccard_coassign"].mean(),
        "median_Jaccard": df["Jaccard_coassign"].median(),
        "IQR_Jaccard": iqr(df["Jaccard_coassign"])
    }
    return pd.Series(summary)


In [ ]:
for K in sorted(clusters_dict.keys()):
    for NN in sorted(clusters_dict[K].keys()):
        print(f"K={K}, n_neighbors={NN}, solutions={clusters_dict[K][NN].keys()}")

In [ ]:
K = 15  # for example
# (1) All-pairs stability at fixed K
#df_pairs = pairwise_metrics_for_K(clusters_dict, K, ignore_noise=True)
#stab_summary = summarize_metrics(df_pairs)

# (2) Reference-vs-others at fixed K
ref_key = (100, 22)  # (n_neighbors, min_samples) for your chosen solution
df_ref = reference_vs_others_for_K(clusters_dict, K, ref_key, ignore_noise=True)
ref_summary = summarize_metrics(df_ref)

In [ ]:
print(df_ref)
print(ref_summary)

plt.plot(df_ref.index, df_ref["ARI"], marker='o', label='Mean ARI')
plt.violinplot(df_ref["ARI"])

## HDBSCAN with UMAP with Projection 

In [ ]:
from sklearn.cluster import HDBSCAN
import pickle

vectors = fv_reduced["l2"]

umap_model = umap.UMAP(
n_components=6,
n_neighbors=100,
min_dist=0,
random_state=42,
metric="cosine"
).fit_transform(vectors)

hdbscan_model = HDBSCAN(min_cluster_size=100, min_samples=22, metric="cosine", store_centers="medoid").fit(umap_model)
hdbscan_model.labels_ = utils.relabel_by_size(hdbscan_model.labels_)

path_to_save = base_dir + '/outputs'
filename = 'hdbscan_l2norm_90pca_6components_100nn_0dist_cosine_42randseed_100_minclustsize_22minsamples.pkl'
with open(os.path.join(path_to_save, filename), 'wb') as file:
    pickle.dump(hdbscan_model, file)

for i in range(-1, max(hdbscan_model.labels_)+1):
    n_points = np.sum(hdbscan_model.labels_ == i)
    print(f"Cluster {i}: {n_points} points")


umap2d = plot_umap(vectors,
                    model_dict,
                    show_images=False,
                    zoom=0.3,
                    n_neighbors=100,
                    min_dist=0)

utils.print_projection(hdbscan_model.labels_, model_dict, umap2d, show_images=False, negative_silhouette=False, clusters_per_marker=None, zoom=0.4)
utils.print_projection(hdbscan_model.labels_, model_dict, umap2d, show_images=True, negative_silhouette=False, clusters_per_marker=None, zoom=0.4)

#utils.print_clusters(hdbscan_model.labels_, vectors, model_dict, random_sample=200, print_filename=False, silhouette=False)

In [ ]:
umap2d = plot_umap(vectors,
                    model_dict,
                    plot_2d=False,
                    show_images=False,
                    zoom=0.3,
                    n_neighbors=100,
                    min_dist=0)

print(np.shape(umap2d))

In [ ]:
import sklearn.cluster
import hdbscan

vectors = fv_reduced["l2"]

umap_model = umap.UMAP(
n_components=6,
n_neighbors=100,
min_dist=0,
random_state=42,
metric="cosine"
).fit_transform(vectors)

# umap_embedding is the 2‑D array returned by UMAP
umap_model = np.asarray(umap_model, dtype=np.float64)

"""# scikit‑learn version: add 1 to min_cluster_size and min_samples
sk_model = sklearn.cluster.HDBSCAN(min_cluster_size=100, min_samples=22,
                                   metric="cosine").fit(umap_model)"""

hdbscan_model= hdbscan.HDBSCAN(min_cluster_size=99,
                    min_samples=21,
                    metric="cosine",
                    algorithm="generic").fit(umap_model)

for i in range(-1, max(hdbscan_model.labels_)+1):
    n_points = np.sum(hdbscan_model.labels_ == i)
    print(f"Cluster {i}: {n_points} points")

"""for i in range(-1, max(sk_model.labels_)+1):
    n_points = np.sum(sk_model.labels_ == i)
    print(f"Cluster {i}: {n_points} points")
    """
print(validity_index(umap_model, hdbscan_model.labels_, metric="cosine"))



In [ ]:
import seaborn as sns
import glasbey
hdbscan_model.condensed_tree_.plot(leaf_separation=1,
select_clusters=True,
label_clusters=True,
log_size=True,
max_rectangles_per_icicle=2)

In [ ]:
print(hdbscan_model.cluster_persistence_)
print(hdbscan_model.labels_)

# Figures


## Imports/Setup

In [ ]:
import os
import pickle
from sklearn.base import clone
from sklearn.cluster import KMeans
from importlib import reload
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.offsetbox import (AnnotationBbox, DrawingArea, OffsetImage,
                                  TextArea)
from matplotlib.textpath import TextPath
from matplotlib.font_manager import FontProperties

# Image Directories
image_dir = 'Images/DM_Images_sorted_cropped_448'

path_to_save = base_dir + '/outputs'

# HDBSCAN model path
hdbscan_path = 'hdbscan_l2norm_90pca_6components_100nn_0dist_cosine_42randseed_100_minclustsize_22minsamples.pkl'

with open(os.path.join(path_to_save, hdbscan_path), 'rb') as file:
    hdbscan_model = pickle.load(file)

hdbscan_model.labels_ = utils.relabel_by_size(hdbscan_model.labels_)

# UMAP embeddings paths
umap_model_path = 'umap_embeddings_6d_100nn.pkl'
umap_2d_path = 'umap_embeddings_2d_100nn.pkl'
umap_2d_150_path = 'umap_embeddings_2d_150nn.pkl'

with open(os.path.join(path_to_save, umap_model_path), "rb") as file:
    umap_model = pkl.load(file)

with open(os.path.join(path_to_save, umap_2d_path), "rb") as file:
    umap_2d = pkl.load(file)

with open(os.path.join(path_to_save, umap_2d_150_path), "rb") as file:
    umap_2d_150 = pkl.load(file)


## Save Cluster Drawings in Folders

In [ ]:
dir = 'clusters'
os.makedirs(dir, exist_ok=False)

model_dict['filenames']
image_dir = model_dict['image_dir']

# Group filenames and distances by cluster
clustered_filenames = {}

# Group filenames by cluster
for filename, label in zip(filenames, hdbscan_model.labels_):
    if label not in clustered_filenames:
        clustered_filenames[label] = []
    clustered_filenames[label].append(filename)

# Sort the dictionaries by the length of the lists in their keys
sorted_clusters = sorted(clustered_filenames.items(), key=lambda item: len(item[1]))

# Group filenames by cluster
for cluster, images in clustered_filenames.items():

    cluster_dir = f'{dir}/cluster_{cluster}'

    os.makedirs(cluster_dir, exist_ok=False)

    for filename in images:
        image = cv2.imread(os.path.join(image_dir, filename))
        cv2.imwrite(f'{cluster_dir}/{filename}', image)

## Display Clusters

In [ ]:
import numpy as np

# Print the size of each cluster in the current k-means clustering result

unique_labels, counts = np.unique(hdbscan_model.labels_, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Cluster {label}: {count} points")

utils.print_clusters(hdbscan_model.labels_, vectors, model_dict, random_sample=100, print_filename=False, silhouette=False)

## Import Labels

In [ ]:
import pandas as pd
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.gridspec as gridspec
from collections import defaultdict

# Labels
labels_path = config["project_paths"]['labels'] # path to the labels file
labels_df = pd.read_excel(labels_path, sheet_name='Sheet1')

labels_df['cluster'] = pd.to_numeric(labels_df['cluster'], errors='coerce').round().astype('Int64')
labels_df = labels_df.dropna(subset=['cluster']).copy()
labels_df['cluster'] = labels_df['cluster'].astype(int)
labels_df['category'] = labels_df['category'].astype(str).str.strip().str.lower()

n_clusters = len(set(hdbscan_model.labels_))

label_order = ['kluverian', 'non-kluverian', 'non-geometric', 'miscellaneous', "noise"]

for label in label_order:
    print(label)
    clusters_for_label = labels_df[labels_df['category'] == label]['cluster'].tolist()
    num_images = labels_df[labels_df['category'] == label]['quantity'].sum()
    print(f'# Clusters = {len(clusters_for_label)}, {len(clusters_for_label)/n_clusters*100:.2f}% of total clusters')
    print(f'# Images = , {num_images}, {num_images/len(filenames)*100:.2f}% of total images')

## Images JSON

In [ ]:
import website
import json as std_json


data = website.build_embeddings_scattergl_demo(
        umap=umap_2d_150,
        labels=hdbscan_model.labels_,
        labels_df=labels_df
        )

with open("dm_data.json", "w") as file:
    string_data = std_json.dumps(data)
    
    file.write(string_data)
print("Data saved to data.json")

### UMAP DataMapPlots

In [ ]:
import datamapplot
import requests
import io

hdbscan_description_labels = []

hdbscan_model.labels_ = utils.relabel_by_size(hdbscan_model.labels_)

for label in hdbscan_model.labels_:
    if label == -1:
        hdbscan_description_labels.append('Unlabelled')
    else:
        hdbscan_description_labels.append(label)
    """else:
        description = labels_df.loc[labels_df['cluster'] == label, 'category']
        hdbscan_description_labels.append(str(description.iloc[0]))"""

print(len(hdbscan_description_labels))

print(umap_model)

datamapplot.create_plot(umap_model,
                        hdbscan_description_labels,
                        color_label_text=False,
                        label_over_points=True,
                        label_font_size=24,
                        )


### UMAP Figure

In [ ]:
utils.print_projection(hdbscan_model.labels_,
                       model_dict,
                       umap2d,
                       show_images=True,
                       with_filenames=True,
                       zoom=0.4)

In [ ]:
utils.pprint_projection(hdbscan_model.labels_,
                       model_dict,
                       umap_2d_150,
                       image_labels="selected",
                       cluster_descriptions=labels_df,
                       hdbscan_model=hdbscan_model,
                       clusters_per_marker=None,
                       zoom=1.6,
                       #probability_desaturate=False,
                       #hdbscan_model=None
                       )


In [ ]:
description = "123,456382349/3333"
bad = {' ': 0, ',': 1, '/': 1}  # value = chars to INCLUDE when breaking at that char
width = 10

def wrap_prefer_breaks(s: str, bad_map={' ': 0, ',': 1, '/': 1}, max_width=10) -> str:
    lines = []
    line_start = 0
    last_break = None       # index of last seen break char
    last_break_keep = 0     # how many chars to include from the break

    for i, ch in enumerate(s):
        # track preferred breakpoints
        if ch in bad_map:
            last_break = i
            last_break_keep = bad_map[ch]

        # if we've exceeded width, break
        if i - line_start + 1 > max_width:
            if last_break is not None and last_break >= line_start:
                cut = last_break + last_break_keep
                lines.append(s[line_start:cut])
                line_start = last_break + 1
                last_break = None
                last_break_keep = 0
            else:
                # no good breakpoint in window: hard break before current char
                lines.append(s[line_start:i])
                line_start = i

    # add the remainder
    if line_start < len(s):
        lines.append(s[line_start:])

    return "\n".join(lines)

description_new = wrap_prefer_breaks(description, bad, width)
print(description_new)


In [ ]:
import glasbey

glasbey.create_palette(palette_size=15, chroma_bounds=(60, 100), lightness_bounds=(30, 80))

### Label Order and Figure Colors

In [ ]:
label_order = ['kluverian', 'non-kluverian', 'non-geometric', 'miscellaneous', "noise"]

background_color = {'kluverian': "#caeafb",
                    'non-kluverian': "#dad6fe",
                    'non-geometric': "#e4fdce",
                    'miscellaneous': "#fcdbcc",
                    'noise': "#dfdede"}


In [ ]:
def plot_label_figs(num_columns=6,
                    num_clust_columns=1,
                    image_selection="medoid",
                    seed=42,
                    include_noise=True):
    """
    image_selection:
      - "medoid"/"mediod": nearest to cluster medoid (first image is medoid)
      - "random": random order, reproducible with `seed`
    Excludes cluster -1.
    """
    # RNG: re-init every call => identical output for same seed
    rng = np.random.default_rng(seed)

    # grid sanity
    assert isinstance(num_columns, int) and num_columns >= 2
    assert isinstance(num_clust_columns, int) and num_clust_columns >= 1

    # labels & filenames
    clusters_ = hdbscan_model.labels_.astype(int)
    filenames = np.asarray(model_dict['filenames'])
    image_dir = model_dict['image_dir']

    # feature space for medoid computation
    try:
        X = umap_model  # embedding array from your Cell 2
    except NameError:
        X = vectors
    if X.shape[0] != len(filenames):
        X = vectors  # fallback if mismatch

    # cluster -> indices (exclude -1)
    cluster_indices = defaultdict(list)
    for i, lbl in enumerate(clusters_):
        if include_noise and lbl == -1:
            cluster_indices[int(lbl)].append(i)
        elif lbl != -1:
            cluster_indices[int(lbl)].append(i)

    # build per-cluster ordered filename lists
    clustered_filenames = {}

    if image_selection == "medoid":
        for lbl in sorted(cluster_indices.keys()):   # sorted => stable
            idxs = cluster_indices[lbl]
            Xi = X[idxs]
            if len(idxs) == 0:
                continue
            # Euclidean medoid
            diff = Xi[:, None, :] - Xi[None, :, :]
            D = np.linalg.norm(diff, axis=2)
            medoid_local = np.argmin(D.sum(axis=1))
            d_to_medoid = D[:, medoid_local]
            order = np.argsort(d_to_medoid)          # medoid first
            clustered_filenames[lbl] = [filenames[idxs[k]] for k in order]
        note_text = "closest to\nmedoid"
    elif image_selection == "random":
        for lbl in sorted(cluster_indices.keys()):   # sorted => stable across calls
            idxs = cluster_indices[lbl]
            if len(idxs) == 0:
                continue
            order = rng.permutation(len(idxs))       # uses our local, re-seeded RNG
            clustered_filenames[lbl] = [filenames[idxs[k]] for k in order]
        note_text = ""
    else:
        raise ValueError("image_selection must be 'medoid'/'mediod' or 'random'")

    # plot: one figure per label (labels_df already sorted by size)
    for category in label_order:
        target_clusters = labels_df.loc[labels_df['category'] == category, 'cluster'].astype(int).tolist()
        target_clusters = [c for c in target_clusters if c != -1 or include_noise]

        num_rows = len(target_clusters)
        if num_rows == 0:
            continue

        fig = plt.figure(figsize=(num_clust_columns * num_columns, num_rows),
                         facecolor=background_color.get(category, "white"))
        outer_grid = gridspec.GridSpec(num_rows, num_clust_columns, wspace=0.02, hspace=0.1)
        clusters_plotted = 0

        for cluster_id in target_clusters:
            images = clustered_filenames.get(cluster_id, [])
            if not images:
                continue

            col = (clusters_plotted // num_rows)
            row = (clusters_plotted % num_rows)
            inner_grid = gridspec.GridSpecFromSubplotSpec(
                1, num_columns, subplot_spec=outer_grid[row, col], wspace=0.1, hspace=0
            )
            clusters_plotted += 1

            for plot_idx in range(num_columns):
                ax = fig.add_subplot(inner_grid[0, plot_idx])

                if clusters_plotted == 1 and plot_idx == 0:
                    if cluster_id != -1:
                        ax.text(0.5, 1.13, "Cluster", fontsize=10, ha='center', va='bottom', fontstyle='italic')

                    ax.text(1.6, 1.1, note_text, fontsize=10, ha='center', va='bottom', fontstyle='italic')

                if plot_idx == 0:
                    dser = labels_df.loc[labels_df['cluster'] == cluster_id, 'description']
                    desc = dser.iloc[0] if not dser.empty else None
                    include_desc = (isinstance(desc, str) and desc.strip() != "") and not pd.isna(desc)

                    text = f"{cluster_id}\nN={len(images)}" #if not include_desc else f"{cluster_id}\n{desc.strip()}\nN={len(images)}"
                    ax.text(0.98, 0.5, text, ha='right', va='center',
                            fontsize=10, transform=ax.transAxes, wrap=True)
                    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
                    ax.margins(0); ax.axis('off')
                else:
                    img_idx = plot_idx - 1
                    if img_idx < len(images):
                        fname = images[img_idx]
                        img = cv2.imread(os.path.join(image_dir, str(fname)))
                        if img is not None:
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                            ax.imshow(img)
                    ax.axis('off')

        plt.show()

plot_label_figs(num_columns=6,
                num_clust_columns=1,
                image_selection="random",
                seed=42)

# Location and Attendance Analysis

In [ ]:
import pickle

# Image Directories
image_dir = 'Images/DM_Images_sorted_cropped_448'

model_name = 'dino'
kmeans = {}

vectors = fv_reduced["l2"]

## Determining Location Frequencies


In [ ]:
import pandas as pd

metadata_path = config["project_paths"]['metadata']
metadata_df = pd.read_excel(metadata_path, sheet_name='Sheet1')

metadata_labeled_path = config["project_paths"]['metadata_labeled']
metadata_labeled_df = pd.read_excel(metadata_labeled_path, sheet_name='Sheet1')

metadata_labeled_df['is_inlier'] = metadata_labeled_df['filename'].isin(model_dict["filenames"])
metadata_df['is_inlier'] = metadata_df['filename'].isin(model_dict["filenames"])

print("NO LOCATION DATA: ", metadata_labeled_df['is_inlier'].value_counts())
print("METADATA: ", metadata_df['is_inlier'].value_counts())

# We are missing metadata for 67 images

location_frequencies = metadata_df[metadata_df['is_inlier'] == True]['location'].value_counts()
location_frequencies_percentage = location_frequencies / location_frequencies.sum() * 100  # Convert to percentage
print("Location Frequencies:\n", location_frequencies)
print("Location Frequencies (Percentage):\n", location_frequencies_percentage)

metadata_df.fillna({"location": 'Unknown'}, inplace=True)
print(metadata_df['location'].value_counts())

## Deterimining Location for Clusters

### Loading and adding clusters to dataframe

In [ ]:


metadata_path = config["project_paths"]['metadata']
metadata_df = pd.read_excel(metadata_path, sheet_name='Sheet1')

not_in_clusterd_filenames = []

new_labels = utils.relabel_by_size(hdbscan_model.labels_)

for filename, label in zip(filenames, new_labels):

    label = int(label)

    metadata_df.loc[metadata_df['filename'] == filename, 'cluster'] = label # add cluster data if filename exists in metadata_df

    # If filename not in metadata_df, add new row with filename and cluster data
    if filename not in metadata_df['filename'].values:
        metadata_df = pd.concat([metadata_df, pd.DataFrame({'filename': [filename], 'cluster': [label]})], ignore_index=True)

metadata_df['cluster'] = metadata_df['cluster'].astype('Int64')

display(metadata_df)

print(metadata_df['cluster'].value_counts(sort=False))
print(metadata_df['cluster'].count())

### Chi Squared Test

The chi-squared p-value will be technically valid only if you treat clusters as if they were fixed categories (like “treatment A vs treatment B”), not if you treat clustering as part of the randomness.
In other words: if your scientific question is “given the clusters we defined, is location independent of cluster?”, then is fine.

In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table of cluster vs location
filtered_df = metadata_df[metadata_df['cluster'].notna()]
contingency = pd.crosstab(filtered_df['cluster'], filtered_df['location'], dropna=False)

# Perform Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency)
print(f"Chi-squared: {chi2}, p-value: {p}, Degrees of freedom: {dof}")


# Create a contingency with totals
contingency_totals = pd.crosstab(filtered_df['cluster'], filtered_df['location'], dropna=False)
contingency_totals['Sum'] = contingency.sum(axis=1)
contingency_totals.loc['Sum'] = contingency.sum(axis=0)
print("\nContingency Table with Totals:")
display(contingency_totals)

# Create a contingency table normalized by rows (percentage values)
percentage_contingency = pd.crosstab(filtered_df['cluster'], filtered_df['location'], dropna=False, normalize='index') * 100
percentage_contingency = percentage_contingency.round(2)

overall_total = metadata_df['cluster'].count()
row_pct = ((contingency.sum(axis=1) / overall_total) * 100).round(2)
col_pct = ((contingency.sum(axis=0) / overall_total) * 100).round(2)
percentage_contingency['RowTotal'] = row_pct
col_pct['Sum'] = 100.0
percentage_contingency.loc['Sum'] = col_pct

print("\nPercentage Contingency Table with Totals:")
display(percentage_contingency)

### Chi Squared Without No location

In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table of cluster vs location
contingency = pd.crosstab(metadata_df['cluster'], metadata_df['location'])

# Perform Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency)
print(f"Chi-squared: {chi2}, p-value: {p}, Degrees of freedom: {dof}")


# Create a contingency with totals
contingency_totals = pd.crosstab(metadata_df['cluster'], metadata_df['location'])
contingency_totals['Sum'] = contingency.sum(axis=1)
contingency_totals.loc['Sum'] = contingency.sum(axis=0)
print("\nContingency Table with Totals:")
display(contingency_totals)

# Create a contingency table normalized by rows (percentage values)
percentage_contingency = pd.crosstab(metadata_df['cluster'], metadata_df['location'], normalize='index') * 100
percentage_contingency = percentage_contingency.round(2)

overall_total = metadata_df['cluster'].count()
row_pct = ((contingency.sum(axis=1) / overall_total) * 100).round(2)
col_pct = ((contingency.sum(axis=0) / overall_total) * 100).round(2)
percentage_contingency['RowTotal'] = row_pct
col_pct['Sum'] = 100.0
percentage_contingency.loc['Sum'] = col_pct

print("\nPercentage Contingency Table with Totals:")
display(percentage_contingency)

### Adjusted Pearson (Standardized) Residuals

Resource: https://cscu.cornell.edu/wp-content/uploads/conttableresid.pdf

In [ ]:
import seaborn as sns
from scipy.stats import norm

# standardized residuals = (observed - expected) / sqrt(expected)
residuals = (contingency - expected) / np.sqrt(expected*(1 - contingency.sum(axis=1).values[:, None]/contingency.values.sum())*(1 - contingency.sum(axis=0).values[None, :]/contingency.values.sum()))

plt.figure(figsize=(6, 8))
sns.heatmap(residuals.round(2), annot=True, cmap='coolwarm')
plt.title("Standardized Residuals Heatmap")
plt.show()

bonferrioni_threshold = 0.05 / (contingency.shape[0] * contingency.shape[1])
critical_value = norm.ppf(1 - bonferrioni_threshold/2)
print(f"Bonferroni-corrected significance threshold: {bonferrioni_threshold}")
print(f"Critical Chi-squared value for Bonferroni correction: {critical_value}")

When testing all combinations of cluster and location, it was seen that images in cluster 1 from London are obseved signficantly more than expected.

### Effect Size - Cramer's V

If V is, say, <0.1, that’s a very weak association — i.e. statistically significant but practically negligible.

“Although the chi² test indicates a significant difference from the global distribution (p=0.006), the effect size is negligible (Cramér’s V=0.046), suggesting the location distribution is approximately stable across clusters.”

In [ ]:
n = contingency.values.sum()
cramer_v = np.sqrt(chi2 / (n * (min(contingency.shape)-1)))
print("Cramer's V:", cramer_v)

## Determining Location Attendance and Session Frequencies

### HS & DL Sessions

In [ ]:
with open("config.yaml", 'r') as f:
        config = yaml.safe_load(f)

session_attendance_path = config["project_paths"]['session_attendance_data']
session_attendance_df = pd.read_excel(session_attendance_path, sheet_name='Sheet1')

# Session counts and percentages
session_counts = session_attendance_df['session'].value_counts()
print("Session Counts:", session_counts)
print("Total:", session_attendance_df['session'].count())
session_percentage = session_counts / session_counts.sum() * 100
print("Session Percentage:")
print(session_percentage)

# Attendance counts by session type
session_type_counts = session_attendance_df.groupby('session')['scanned'].sum()
print("\nAttendance per Session Counts:", session_type_counts)
print("Total:", session_attendance_df['scanned'].count())
print("Attendance per Session Percentage:")
print(session_type_counts / session_attendance_df['scanned'].sum() * 100)

### By Location

In [ ]:
# Session counts and percentages
location_counts = session_attendance_df['location'].value_counts()
print("Location Counts:", location_counts)
print("Total:", session_attendance_df['location'].count())
location_percentage = location_counts / location_counts.sum() * 100
print("Location Percentage:")
print(location_percentage)


# Attendance counts by session type
session_type_counts = session_attendance_df.groupby('location')['scanned'].sum()
print("\nAttendance per Session Counts:", session_type_counts)
print("Total:", session_attendance_df['scanned'].count())
print("Attendance per Session Percentage:")
print(session_type_counts / session_attendance_df['scanned'].sum() * 100)